<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/ActualGraphic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TODO

* парсинг таблицы ГРАФИК ЗАКАЗЧИКА из АСС
* парсинг договора подрядчика и автоматическая генерация ГУГЛ таблиц
* каждому виду работ в контрактном графике необходимо присвоить НАШ шифр
* график заказчика подгружается с локального компьютера и сохраняется туда же
* приветсти все данные к НОРМАЛЬНОМУ ТИПУ, включая индексы и названия колонок
* разобраться с процентами

# Библиотеки

In [ ]:
import openpyxl
from openpyxl.cell.cell import WriteOnlyCell
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook

import pandas as pd

# Предобработка графика работ подрядчика

* предварительно необходимо привести график в гугл форму
* необходимо предоставить доступ корректировщику

In [ ]:
from google.colab import auth
auth.authenticate_user()
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime
worksheet = gc.open('План работ Договор Адамант')

In [ ]:
graf = worksheet.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])
graf

In [ ]:
graf.iloc[:, 8].values

# Предобработка графика Заказчика

In [ ]:
# файл загружается на ГУГЛ диск
from google.colab import files
upload = files.upload()

Saving Сводный план-график 2022 12 28 иса.xlsx to Сводный план-график 2022 12 28 иса.xlsx


In [ ]:
# содежание текущей папки на ГУГЛ диске
!ls

 sample_data  'Сводный план-график 2022 12 11 иса (1).xlsx'


ПАНДАС из редакции графика **заказчика**

In [ ]:
df_graph = pd.read_excel(open('Сводный план-график 2022 12 11 иса (1).xlsx', 'rb'))

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
idx = pd.IndexSlice

In [ ]:
df_graph = pd.DataFrame(df_graph.values, columns=df_graph.values[3])
df_graph['Название задачи'] = df_graph['Название задачи'].map(lambda z: str(z).lstrip()).values
df_graph

In [ ]:
df_graph.drop([0,1,2,3], inplace=True)

Список задач из графика заказчика

# Объедение списков работ

In [ ]:
#  список из плана работ
list1 = graf.iloc[:, 1].values
list1

In [ ]:
#  список из графика заказчика
list2 = df_graph['Название задачи']
list2 = list2.map(lambda z: str(z).lstrip())
list2 = list2.to_list()
list2

Пересечение списков задач плана работ и графика заказчика

In [ ]:
result = list(set(list1) & set(list2))
result

Обновляем данные в плане заказчика

In [ ]:
for i in result:
    df_graph.loc[df_graph['Название задачи'].map(lambda z: str(z).lstrip()).values == i, '% '] = pd.to_numeric(graf.loc[graf.iloc[:,1].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Название задачи'].map(lambda z: str(z).lstrip()).values == i, 'Название задачи'].values[0]: pd.to_numeric(graf.loc[graf.iloc[:,1].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100})
    

In [ ]:
df_graph.loc[df_graph['Название задачи'] == 'Земляные работы']

,Внесено изменение,СДР,Название задачи,%,План %,Откл. План%,Столбец2,Столбец1,Начало,Окончание,Отв. Исполнитель,Названия ресурсов,NaN,NaN
869,NaN,13.6.3.6,Земляные работы,0.26,28,2,Вт 15.11.22,Сб 18.02.23,2022-11-15 00:00:00,2023-02-18 00:00:00,NaN,2 Уровень. Суммарная задача,NaN,NaN


Вносим данные в файл ексель

In [ ]:
df_graph.to_excel('Сводный план-график {} иса.xlsx'.format('26.12.2022'), sheet_name='Обновление от {}'.format('26.12.2022'),index=False)